In [1]:
import open3d as o3d
import numpy as np
import os
import gen_utils as gu
import pyvista as pv
from time import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
challenge_path = '../datasets/3D_scans_per_patient_obj_files'
challenge_gt_path = '../datasets/ground-truth_labels_instances'
case = '01AS72AN'
jaw = 'lower'

osstem_mesh_path = './test_input_aligned/Case_01_lower(Antagonist)_1.obj'
# osstem_mesh_path = './test_input_aligned/Case_01_upper_1.obj'
# osstem_mesh_path = './test_input_aligned/Case_03_lower(Antagonist)_1.obj'
# osstem_mesh_path = './test_input_aligned/Case_03_upper_1.obj'

In [3]:
challenge_mesh_path = os.path.join(challenge_path, case, case+'_'+jaw+'.obj')
print(challenge_mesh_path)

json_path = os.path.join(challenge_gt_path, case, case+'_'+jaw+'.json')
print(json_path)

../datasets/3D_scans_per_patient_obj_files/01AS72AN/01AS72AN_lower.obj
../datasets/ground-truth_labels_instances/01AS72AN/01AS72AN_lower.json


# Open3d

In [4]:
vertices, org_mesh = gu.read_txt_obj_ls(challenge_mesh_path, ret_mesh=True, use_tri_mesh=True)

org_mesh.compute_vertex_normals()

# gt_loaded_json = gu.load_json(os.path.join(challenge_gt_path, case, case+'_'+jaw+'.json'))
# gt_labels = np.array(gt_loaded_json['labels']).reshape(-1)

# if gt_loaded_json['jaw'] == 'lower':
#     gt_labels -= 20
# gt_labels[gt_labels//10==1] %= 10
# gt_labels[gt_labels//10==2] = (gt_labels[gt_labels//10==2]%10) + 8
# gt_labels[gt_labels<0] = 0

o3d.visualization.draw_geometries([org_mesh])

# colored_mesh = gu.get_colored_mesh(org_mesh, gt_labels)

# o3d.visualization.draw_geometries([colored_mesh])

## open3d.geometry.sample_points_poisson_disk

In [56]:
np.array(colored_mesh.vertex_colors).dtype

0.00020575523376464844

In [57]:
start = time()
pcd = org_mesh.sample_points_poisson_disk(24000)
end = time()

o3d.visualization.draw_geometries([pcd])
end-start

5.972134828567505

In [7]:
pcd.points

std::vector<Eigen::Vector3d> with 24000 elements.
Use numpy.asarray() to access data.

In [8]:
pcd.colors

std::vector<Eigen::Vector3d> with 24000 elements.
Use numpy.asarray() to access data.

In [9]:
color_ind = np.array(pcd.colors)
color_ind.dtype

dtype('float64')

In [10]:
palet = np.array([
        [255,153,153],

        [153,76,0],
        [153,153,0],
        [76,153,0],
        [0,153,153],
        [0,0,153],
        [153,0,153],
        [153,0,76],
        [64,64,64],

        [255,128,0],
        [153,153,0],
        [76,153,0],
        [0,153,153],
        [0,0,153],
        [153,0,153],
        [153,0,76],
        [64,64,64],
    ])/255
palet[9:] *= 0.4

# palet = np.array([
#         [0, 0, 0],
        
#         [10, 10, 10],
#         [20, 20, 20],
#         [30, 30, 30],
#         [40, 40, 40],
#         [50, 50, 50],
#         [60, 60, 60],
#         [70, 70, 70],
#         [80, 80, 80],
        
#         [90, 90, 90],
#         [100, 100, 100],
#         [110, 110, 110],
#         [120, 120, 120],
#         [130, 130, 130],
#         [140, 140, 140],
#         [150, 150, 150],
#         [160, 160, 160]
# ])

In [11]:
palet.dtype

dtype('float64')

In [12]:
sampled_label = np.zeros(24000)
for i, p in enumerate(palet):
    sampled_label[((np.isclose(color_ind, p)).sum(-1)//3).astype(np.bool_)] = i

In [13]:
for i in range(17):
    print(len(sampled_label[sampled_label==i]))

20323
178
188
245
237
237
435
353
0
177
224
240
218
208
401
336
0


In [14]:
print(len(color_ind[sampled_label==-1]))
color_ind[sampled_label==-1].astype(np.float64)

0


array([], shape=(0, 3), dtype=float64)

In [15]:
vertices, org_mesh = gu.read_txt_obj_ls(mesh_path, ret_mesh=True, use_tri_mesh=False)

cloud = pv.PolyData(np.array(pcd.points))
mesh = cloud.delaunay_2d()

org_mesh.vertices = o3d.utility.Vector3dVector(np.array(mesh.points))
org_mesh.triangles = o3d.utility.Vector3iVector(np.array(mesh.regular_faces))
org_mesh.compute_vertex_normals()

gu.print_3d(gu.get_colored_mesh(org_mesh, sampled_label))

In [16]:
np.array(pcd.normals).shape

(24000, 3)

## open3d.geometry.voxel_down_sample

In [61]:
vertices, org_mesh = gu.read_txt_obj_ls(challenge_mesh_path, ret_mesh=True, use_tri_mesh=True)
vertices.shape

(113080, 6)

In [62]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(vertices[:, :3]))
o3d.visualization.draw_geometries([pcd])

In [63]:
sorted(pcd.compute_nearest_neighbor_distance())[:10], sorted(pcd.compute_nearest_neighbor_distance())[-10:]

([0.0011825935361760406,
  0.0011825935361760406,
  0.0016410329603897648,
  0.0016410329603897648,
  0.002311960429093609,
  0.002311960429093609,
  0.0026146584176523994,
  0.0026146584176523994,
  0.0027376453954646067,
  0.0027376453954646067],
 [4.460783523945331,
  4.460783528249087,
  4.493911321230188,
  4.493911321230188,
  4.493913914127489,
  4.512324044190266,
  4.512324044190266,
  4.512324095029833,
  4.5826826959758105,
  4.706616869999504])

In [22]:
voxel_size = 2
sorted(pcd.voxel_down_sample(voxel_size).compute_nearest_neighbor_distance())[:10], \
sorted(pcd.voxel_down_sample(voxel_size).compute_nearest_neighbor_distance())[-10:]

([0.12012288263725508,
  0.12012288263725508,
  0.26057311027349944,
  0.26057311027349944,
  0.2932751962443689,
  0.2932751962443689,
  0.3264274649051962,
  0.3264274649051962,
  0.342241693478084,
  0.342241693478084],
 [1.9998505721845266,
  2.014923012802051,
  2.021760026912043,
  2.0218952239080745,
  2.041232459692378,
  2.0433228495360494,
  2.0438002424089836,
  2.0522900526425993,
  2.3269442952375994,
  3.287307797132058])

In [64]:
start = time()
pcd.voxel_down_sample(24000)
end = time()
end-start

0.010190248489379883

In [23]:
np.array(pcd.voxel_down_sample(voxel_size).points[:10])

array([[ -15.51582336,   -2.25549984, -111.22479757],
       [  18.79205598,    4.54199908, -113.62487115],
       [  17.19524231,    5.84844799, -110.54884186],
       [ -39.35035801,    6.09764802, -105.38248634],
       [ -37.82429211,    2.43087677, -105.60689955],
       [ -36.82312965,    2.32092181, -105.13002777],
       [ -38.16914153,    4.23025407, -105.44532299],
       [ -39.88845481,    8.77549411, -104.42748999],
       [ -37.10402679,   -1.2281599 , -105.70759583],
       [ -36.94787979,    4.81356382, -104.89863586]])

In [24]:
for p in np.array(pcd.voxel_down_sample(voxel_size).points):
    if p in np.array(pcd.points):
        print(p)

[ -37.10402679   -1.2281599  -105.70759583]
[ -36.94787979    4.81356382 -104.89863586]
[-33.08174133   6.98305511 -96.88136292]
[ -33.03694153   -4.78202915 -102.81986237]
[ -28.73965263  -16.99838448 -105.00170135]
[ -28.81942368  -10.71192074 -103.09117889]
[ -26.94640732  -18.78615379 -102.68399048]
[-32.9794426   13.25167465 -99.11973572]
[ -32.81850433   12.96056652 -103.03846741]
[-32.9499054    1.9546963  -96.61200714]
[-30.88968277  -2.02693605 -93.01068115]
[ -27.02456284  -20.87066269 -103.83361053]
[ -22.83383179  -20.72487259 -100.9759903 ]
[  -1.84245872  -18.78521538 -103.15196991]
[   1.04305792  -14.42703724 -108.63247681]
[  11.25070667  -12.96564198 -104.96987915]
[ -17.54743004   -4.85188818 -113.04679871]
[  19.33495712  -18.90796661 -102.94652557]
[ 17.04068375 -10.5834856  -94.40866852]
[ -22.84854698  -20.69566917 -100.41513824]
[ -18.93487167   -8.27371502 -102.82837677]
[ -22.59011269    5.2655201  -115.3766098 ]
[  22.90984154  -14.65139771 -105.07077026]
[  

In [25]:
np.array(pcd.points).mean(0)

array([ -0.28239968,  -8.01873745, -99.03307996])

In [26]:
o3d.visualization.draw_geometries([pcd.voxel_down_sample(voxel_size)])

In [129]:
np.array(pcd.points)[:10],  \
np.array((pcd.voxel_down_sample(0.1).points))[:10]

(array([[  32.23250961,   24.49151039, -111.03425598],
        [  32.79378128,   24.41981888, -111.25597382],
        [  32.48640823,   24.59178162, -111.42739105],
        [  32.45804977,   24.29204369, -110.77379608],
        [  32.09769821,   24.63780975, -111.29608154],
        [  32.04803848,   24.30966949, -110.60123444],
        [  32.79964828,   24.11439323, -110.6783905 ],
        [  31.68505669,   24.62940979, -111.09718323],
        [  32.43758774,   23.97652054, -110.2582016 ],
        [  33.24604797,   24.25520706, -111.28716278]]),
 array([[  -9.42953491,  -29.64219856, -102.44727325],
        [  -9.69323063,  -29.69818878, -102.41346741],
        [  -9.93113232,  -29.51582909, -102.50177002],
        [   0.42994091,  -29.39485741, -102.46543884],
        [   0.21797167,  -29.37998962, -102.33530426],
        [  -0.26125661,  -29.088974  , -102.40115356],
        [ -15.63829136,   -2.15891051, -111.1860199 ],
        [ -15.49253941,   -2.33407664, -111.36514282],
        

## open3d.geometry.TirangleMesh.simplify_vertex_clustering()

## open3d.geometry.SimplificationContraction.Average

In [65]:
voxel_size = max(org_mesh.get_max_bound() - org_mesh.get_min_bound()) / 128
print(f'voxel_size = {voxel_size:e}')
mesh_smp = org_mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

voxel_size = max(org_mesh.get_max_bound() - org_mesh.get_min_bound()) / 256
print(f'voxel_size = {voxel_size:e}')
start = time()
mesh_smp = org_mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
end = time()
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])
end-start

voxel_size = 6.434156e-01
Simplified mesh has 8890 vertices and 18194 triangles
voxel_size = 3.217078e-01
Simplified mesh has 29322 vertices and 59270 triangles


0.20023703575134277

In [53]:
voxel_size = 1
print(f'voxel_size = {voxel_size:e}')
mesh_smp = org_mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(mesh_smp.vertices)[:, :3])
o3d.visualization.draw_geometries([pcd])

voxel_size = 1.000000e+00
Simplified mesh has 4107 vertices and 8399 triangles


## open3d.geometry.SimplificationContraction.Quadric

In [66]:
voxel_size = max(org_mesh.get_max_bound() - org_mesh.get_min_bound()) / 128
print(f'voxel_size = {voxel_size:e}')
mesh_smp = org_mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Quadric)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

voxel_size = max(org_mesh.get_max_bound() - org_mesh.get_min_bound()) / 256
print(f'voxel_size = {voxel_size:e}')
start = time()
mesh_smp = org_mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Quadric)
end = time()
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])
end-start

voxel_size = 6.434156e-01
Simplified mesh has 8890 vertices and 18194 triangles
voxel_size = 3.217078e-01
Simplified mesh has 29322 vertices and 59270 triangles


0.7241568565368652

In [67]:
voxel_size = 1
print(f'voxel_size = {voxel_size:e}')
mesh_smp = org_mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Quadric)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(mesh_smp.vertices)[:, :3])
o3d.visualization.draw_geometries([pcd])

voxel_size = 1.000000e+00
Simplified mesh has 4107 vertices and 8399 triangles


## open3d.geometry.TriangleMesh.simplify_quadric_decimation

In [79]:
mesh_smp = org_mesh.simplify_quadric_decimation(target_number_of_triangles=1700)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

mesh_smp = org_mesh.simplify_quadric_decimation(target_number_of_triangles=6500)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

mesh_smp = org_mesh.simplify_quadric_decimation(target_number_of_triangles=10000)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp])

Simplified mesh has 927 vertices and 1700 triangles
Simplified mesh has 3426 vertices and 6500 triangles
Simplified mesh has 5236 vertices and 10000 triangles


In [82]:
start = time()
quad = org_mesh.simplify_quadric_decimation(24000)
end = time()
o3d.visualization.draw_geometries([quad])
end - start

3.0823400020599365

# PyMeshLab

In [4]:
import pymeshlab

In [5]:
osstem_mesh = pymeshlab.MeshSet()
challenge_mesh = pymeshlab.MeshSet()

In [6]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
challenge_mesh.load_new_mesh(challenge_mesh_path)

## generate_iso_parametrization_remeshing

In [9]:
challenge_mesh.iso_parametrization_main()
challenge_mesh.generate_iso_parametrization_remeshing()

challenge_mesh.show_polyscope()

/home/pangwoon/anaconda3/envs/roitr/lib/python3.8/site-packages/pymeshlab/__init__.py:41: UserWarning: Deprecated Filter Warning: You should use 'compute_iso_parametrization' instead of 'iso_parametrization_main'. See https://pymeshlab.readthedocs.io/en/latest/index.html#filters-renaming
  res_dict = self.apply_filter(name, **kwargs)



 STEP 1 


PyMeshLabException: Failed to apply filter: iso_parametrization_main
Details: non possible parameterization because of non manifold edges

## ~~generate_marching_cubes_apss~~

In [10]:
challenge_mesh.generate_marching_cubes_apss()

challenge_mesh.show_polyscope()

## ~~generate_resampled_uniform_mesh~~

In [7]:
challenge_mesh.generate_resampled_uniform_mesh()

In [11]:
challenge_mesh.generate_resampled_uniform_mesh(offset=pymeshlab.Percentage(80))

In [8]:
challenge_mesh.generate_resampled_uniform_mesh(cellsize=pymeshlab.Percentage(1.25), offset=pymeshlab.Percentage(80))

In [13]:
challenge_mesh.generate_resampled_uniform_mesh(cellsize=pymeshlab.Percentage(1.25), offset=pymeshlab.Percentage(80))

In [23]:
challenge_mesh.generate_resampled_uniform_mesh(cellsize=pymeshlab.Percentage(0.01))

In [ ]:
challenge_mesh.show_polyscope()

## generate_sampling_clustered_vertex

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_clustered_vertex(sampling=1)

In [8]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_clustered_vertex(sampling=0)

In [9]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_clustered_vertex(threshold=pymeshlab.Percentage(0.1), sampling=0)

In [19]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
osstem_mesh.generate_sampling_clustered_vertex(sampling=0)

In [12]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
osstem_mesh.generate_sampling_clustered_vertex(threshold=pymeshlab.Percentage(0.1), sampling=0)

In [14]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
osstem_mesh.generate_sampling_clustered_vertex(threshold=pymeshlab.Percentage(0.5), sampling=0)

In [29]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
osstem_mesh.generate_sampling_clustered_vertex(threshold=pymeshlab.Percentage(0.5), sampling=1)

In [10]:
osstem_mesh.show_polyscope()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 470.103.01


## ~~generate_sampling_element~~

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_element(sampling=0, samplenum=24000)

In [8]:
challenge_mesh.show_polyscope()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 470.103.01


## ~~generate_sampling_montecarlo~~

In [9]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_montecarlo(samplenum=24000)

In [10]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_montecarlo(samplenum=24000, radiusvariance=0.5)

In [11]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_montecarlo(samplenum=24000, radiusvariance=0.5, exactnum=False)

In [12]:
challenge_mesh.show_polyscope()

## generate_sampling_poisson_disk

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_poisson_disk(samplenum=24000)

In [8]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_poisson_disk(samplenum=24000, radius=pymeshlab.Percentage(1))

In [9]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_poisson_disk(samplenum=24000, radius=pymeshlab.Percentage(1), approximategeodesicdistance=True)

In [13]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_poisson_disk(samplenum=24000, radius=pymeshlab.Percentage(1), approximategeodesicdistance=True, exactnumflag=False)

In [11]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_sampling_poisson_disk(samplenum=24000, radius=pymeshlab.Percentage(0.5), approximategeodesicdistance=True, exactnumflag=True)
end = time()
end - start

7.525360107421875

In [12]:
challenge_mesh.show_polyscope()

In [7]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
start = time()
osstem_mesh.generate_sampling_poisson_disk(samplenum=24000, radius=pymeshlab.Percentage(0.5), approximategeodesicdistance=True, exactnumflag=False)
end = time()
end - start

0.4485130310058594

In [8]:
osstem_mesh.show_polyscope()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 470.103.01


## ~~generate_sampling_regular_recrusive~~

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_regular_recursive()

In [8]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_regular_recursive(cellsize=pymeshlab.Percentage(1))

## ~~generate_sampling_stratifies_triangle~~

In [10]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_stratified_triangle()

In [11]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_stratified_triangle(samplenum=24000)

In [12]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_stratified_triangle(samplenum=24000, sampling=1)

In [13]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_stratified_triangle(samplenum=24000, sampling=3)

In [14]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
challenge_mesh.generate_sampling_stratified_triangle(samplenum=24000, sampling=4)

## ~~generate_sampling_volumetric~~

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_sampling_volumetric()
end = time()
end - start

Surface Sampling radius 0.220014 - montecarlo 1967691vn - Poisson 132002vn
Made 428895 Trials to get 200000 samples


12.829285621643066

## ~~generate_sampling_voronoi~~

In [9]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_sampling_voronoi()
end = time()
end - start

7.827391862869263

In [9]:
challenge_mesh.load_new_mesh(osstem_mesh_path)
start = time()
challenge_mesh.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.3), exactnumflag=True)
end = time()
end - start

0.07111048698425293

## **generate_simplified_point_cloud**

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.01), exactnumflag=True)
challenge_mesh.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.3), exactnumflag=True)
end = time()
end - start

0.3111603260040283

In [18]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.2), exactnumflag=True)
end = time()
end - start

0.261340856552124

In [8]:
challenge_mesh.show_polyscope()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 470.103.01


In [10]:
challenge_mesh.show_polyscope()

In [7]:
palet = np.array([
            [255,153,153],

            [153,76,0],
            [153,153,0],
            [76,153,0],
            [0,153,153],
            [0,0,153],
            [153,0,153],
            [153,0,76],
            [64,64,64],

            # [255,128,0],
            # [153,153,0],
            # [76,153,0],
            # [0,153,153],
            # [0,0,153],
            # [153,0,153],
            # [153,0,76],
            # [64,64,64],
            [20, 10, 0],
            [10, 10, 0],
            [10, 20, 0],
            [0, 10, 20],
            [0, 0, 20],
            [10, 0, 10],
            [10, 0, 0],
            [10, 10, 10],
            ])/255
# palet[9:] *= 0.5

In [8]:
loaded_json = gu.load_json(json_path)
labels = np.array(loaded_json['labels']).reshape(-1)
if loaded_json['jaw'] == 'lower':
    labels -= 20
labels[labels//10==1] %= 10
labels[labels//10==2] = (labels[labels//10==2]%10) + 8
labels[labels<0] = 0

labels

array([ 0,  0,  0, ...,  2, 15, 14])

In [9]:
set(labels)

{0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15}

In [20]:
label_colors = np.zeros(challenge_mesh[0].vertex_matrix().shape)

for idx, p in enumerate(palet):
    label_colors[labels==idx] = palet[idx]
    
label_colors = np.concatenate([label_colors, np.ones((label_colors.shape[0], 1))], axis=-1)
    
label_colors.shape

(113080, 4)

In [21]:
challenge_mesh[0].vertex_matrix().shape, challenge_mesh[0].vertex_scalar_array().shape, label_colors.shape, challenge_mesh[0].vertex_normal_matrix().shape

((113080, 3), (113080,), (113080, 4), (113080, 3))

In [25]:
challenge_mesh[0].vertex_scalar_array()[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
challenge_mesh[0].vertex_normal_matrix().shape

(113080, 3)

In [36]:
new_mesh = pymeshlab.Mesh(vertex_matrix = challenge_mesh[0].vertex_matrix(),
                          v_normals_matrix = challenge_mesh[0].vertex_normal_matrix(),
                          # v_scalar_array = challenge_mesh[0].vertex_scalar_array(),
                          # # f_scalar_array = challenge_mesh[0].face_scalar_array(),
                          v_color_matrix = label_colors)

new_mesh

In [38]:
new_mesh.vertex_normal_matrix()[:10]

array([[ 0.18116802, -3.0961297 ,  0.48165861],
       [ 0.36233604, -6.19225942,  0.96331715],
       [ 0.2665178 , -4.00983588,  3.4033412 ],
       [ 0.36233603, -6.19225941,  0.96331716],
       [ 0.14693404,  0.97397437,  6.20549772],
       [ 0.14693404,  0.97397436,  6.20549772],
       [ 0.14693404,  0.97397437,  6.20549772],
       [ 0.14693404,  0.97397437,  6.20549772],
       [ 0.14693405,  0.97397437,  6.20549772],
       [ 0.14693404,  0.97397437,  6.20549772]])

In [39]:
new_mesh_set = pymeshlab.MeshSet()
new_mesh_set.add_mesh(new_mesh)
new_mesh_set.show_polyscope()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 470.103.01
[polyscope] [ERROR] name must not be the empty string


In [40]:
new_mesh_set.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.3), exactnumflag=True)
new_mesh_set.show_polyscope()

[polyscope] [ERROR] name must not be the empty string


In [15]:
len(new_mesh_set)

2

In [16]:
new_mesh_set[1].vertex_matrix().shape

(14674, 3)

In [16]:
new_mesh_set[1].vertex_color_matrix()[:,:3].dtype

dtype('float64')

In [41]:
for i, p in enumerate(palet):
    print((np.isclose(new_mesh_set[1].vertex_color_matrix()[:,:3], p).sum(-1)//3).astype(np.bool_).sum())

6191
424
458
574
544
557
959
830
0
434
517
565
511
483
874
753
0


In [42]:
sampled_label = np.zeros(new_mesh_set[1].vertex_normal_matrix().shape[0])

for i, p in enumerate(palet):
    sampled_label[((np.isclose(new_mesh_set[1].vertex_color_matrix()[:,:3], p)).sum(-1)//3).astype(np.bool_)] = i

sampled_label.shape

(14674,)

In [19]:
max(sampled_label)

15.0

In [20]:
new_mesh_set[1].vertex_matrix().shape, new_mesh_set[1].face_matrix().shape

((14674, 3), (0, 3))

In [21]:
new_mesh_set[1].vertex_matrix().shape

(14674, 3)

In [22]:
new_mesh_set[1].vertex_color_matrix()[:,:3].shape

(14674, 3)

In [17]:
new_mesh_set[1].vertex_normal_matrix().shape

(14674, 3)

In [18]:
new_mesh_set[1].vertex_normal_matrix()[:10]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [23]:
sampled_label.shape

(14674,)

In [139]:
palet[sampled_label.astype(np.int64)].shape

(14699, 3)

In [143]:
org_mesh = o3d.geometry.PointCloud()
org_mesh.points = o3d.utility.Vector3dVector(new_mesh_set[1].vertex_matrix())
org_mesh.colors = o3d.utility.Vector3dVector(new_mesh_set[1].vertex_color_matrix()[:,:3])

o3d.visualization.draw_geometries([org_mesh])

In [145]:
org_mesh = o3d.geometry.PointCloud()
org_mesh.points = o3d.utility.Vector3dVector(new_mesh_set[1].vertex_matrix())
org_mesh.colors = o3d.utility.Vector3dVector(palet[sampled_label.astype(np.int64)])

o3d.visualization.draw_geometries([org_mesh])

In [74]:
new_mesh.vertex_color_matrix()

array([[1.        , 0.6       , 0.6       , 1.        ],
       [1.        , 0.6       , 0.6       , 1.        ],
       [1.        , 0.6       , 0.6       , 1.        ],
       ...,
       [0.6       , 0.6       , 0.        , 1.        ],
       [0.23921569, 0.        , 0.11764706, 1.        ],
       [0.23921569, 0.        , 0.23921569, 1.        ]])

In [20]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
start = time()
osstem_mesh.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.3), exactnumflag=True)
end = time()
end - start

0.0778205394744873

In [22]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
start = time()
osstem_mesh.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.2), exactnumflag=True)
end = time()
end - start

0.08637094497680664

In [23]:
osstem_mesh.show_polyscope()

## ~~generate_surface_reconstruction_ball_pivoting~~

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_surface_reconstruction_ball_pivoting()
end = time()
end - start

0.8293983936309814

In [9]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_surface_reconstruction_screened_poisson()
end = time()
end - start

10.507348775863647

In [11]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.generate_voronoi_scaffolding()
end = time()
end - start

Surface Sampling radius 1.100069 - montecarlo 78707vn - Poisson 5250vn
Made 214209 Trials to get 100000 samples


## meshing_decimation_clustering

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_decimation_clustering()
end = time()
end - start

0.1861424446105957

In [8]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_decimation_clustering(threshold=pymeshlab.Percentage(0.5))
end = time()
end - start

0.2962193489074707

In [9]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_decimation_clustering(threshold=pymeshlab.Percentage(0.2))
end = time()
end - start

0.7021522521972656

## ~~meshing_decimation_quadric_edge_collapse~~

In [10]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_decimation_quadric_edge_collapse()
end = time()
end - start

2.2861716747283936

In [ ]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_decimation_quadric_edge_collapse()
end = time()
end - start

## **meshing_isotropic_explicit_remeshing**

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_isotropic_explicit_remeshing()
end = time()
end - start

2.625397205352783

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_isotropic_explicit_remeshing(targetlen=pymeshlab.Percentage(0.5))
end = time()
end - start

3.4878664016723633

In [10]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_isotropic_explicit_remeshing(maxsurfdist=pymeshlab.Percentage(0.5))
end = time()
end - start

11.800923585891724

In [7]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_isotropic_explicit_remeshing(iterations=1, adaptive=True, targetlen=pymeshlab.Percentage(1))
end = time()
end - start

20.91249942779541

In [8]:
challenge_mesh.load_new_mesh(challenge_mesh_path)
start = time()
challenge_mesh.meshing_isotropic_explicit_remeshing(iterations=1, adaptive=True, targetlen=pymeshlab.Percentage(0.5))
end = time()
end - start

19.120285511016846

iteration, featuredeg(샘플링 각도 제한)

In [9]:
challenge_mesh.show_polyscope()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 470.103.01


In [9]:
osstem_mesh.load_new_mesh(osstem_mesh_path)
start = time()
osstem_mesh.meshing_isotropic_explicit_remeshing(adaptive=True,featuredeg=1)
end = time()
end - start

0.487562894821167

In [10]:
osstem_mesh.show_polyscope()